#**FACE RECOGNITION**

---
Nna

[1] IEEE |  [Face Recognition using Back Propagation Neural
Network Technique](https://ieeexplore.ieee.org/document/7164700) </br>
[2] Machine Learning Mastery - Jason Brownlee | [Weight Initialization for Deep Learning Neural Networks](https://machinelearningmastery.com/weight-initialization-for-deep-learning-neural-networks/)

#**ALGORITMA**
---


1.   Ekstrak "*face value*" untuk dataset. </br>
    - Greyscale
    - Resize
    - Flatten + Normalisasi
2.   Pemrosesan dataset dengan BPNN</br>
    - **Vektoral**
    - Matriks



##**Import Libraries**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import math
import random
import string
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import os
#import time
#start_time = time.time()
#import matplotlib.pyplot as plt
import hashlib
from collections import defaultdict

##**Data Set**

In [3]:
# Load dataset flatten
dataset = np.load("/content/drive/My Drive//FaceRec_NN/Data_Flatten/Flattened_dataset.npz")
data = dataset['images']
labels = dataset['labels']

print(f"Data shape: {data.shape}, Labels shape: {labels.shape}")

Data shape: (1200, 900), Labels shape: (1200, 15)


In [4]:
columns = [f'X_{i}' for i in range(data.shape[1])] + [f'T_{i}' for i in range(labels.shape[1])]
data_with_labels = np.column_stack([data, labels])

df_dataset = pd.DataFrame(data_with_labels, columns=columns)
df_dataset

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,T_5,T_6,T_7,T_8,T_9,T_10,T_11,T_12,T_13,T_14
0,0.462745,0.858824,0.984314,0.894118,0.929412,0.913725,0.847059,0.905882,0.850980,0.878431,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.788235,0.807843,0.705882,0.792157,0.807843,0.827451,0.815686,0.835294,0.827451,0.831373,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.772549,0.741176,0.741176,0.729412,0.839216,0.815686,0.847059,0.870588,0.874510,0.858824,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.741176,0.623529,0.686275,0.588235,0.647059,0.654902,0.501961,0.458824,0.662745,0.745098,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.800000,0.745098,0.600000,0.600000,0.552941,0.600000,0.560784,0.356863,0.474510,0.698039,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,0.258824,0.298039,0.262745,0.678431,0.854902,0.862745,0.866667,0.815686,0.835294,0.835294,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1196,0.827451,0.835294,0.843137,0.843137,0.839216,0.835294,0.839216,0.843137,0.835294,0.882353,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1197,0.823529,0.827451,0.831373,0.835294,0.839216,0.835294,0.831373,0.827451,0.827451,0.819608,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1198,0.180392,0.231373,0.270588,0.223529,0.172549,0.211765,0.239216,0.192157,0.168627,0.168627,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


##**Implementasi Kode**

**Fungsi Aktivasi**

In [5]:
# Fungsi: sigmoid
def sigmoid(x):
    return 1.0/(1+ np.exp(-x))

def d_sigmoid(x):
    return x * (1.0 - x)


# Fungsi: reLU
def relu(x):
    return np.maximum(0, x)

def d_relu(x):
    return np.where(x > 0, 1, 0)


# Fungsi: softmax
def softmax(x):
    exps = np.exp(x - np.max(x))
    return exps / np.sum(exps)

def d_softmax(x):
    s = softmax(x)
    return s * (1.0 - s)

**Insialisasi**

Parameter

In [16]:
epoch = 100
lr = 0.5

NI = 900             # input neuron (node)
NH = 3               # hidden neuron (node)
hiddenLayer = 1
NO = 15              # output neuron (node)

Bobot (Weight)

In [20]:
def weightsd(NI, NH, NO):
    """
    Fungsi rentang bobot melalui He Weight Initializaiton
    untuk fungsi aktivasi ReLU

    """
    np.random.seed(42)
    std = math.sqrt(2.0 / NI)                   # standar deviasi bobot

    v_weights = np.random.randn(NI, NH) * std   # bobot input-hidden
    vb_weights = np.random.randn(NH) * std   # bobot bias-hidden
    w_wights = np.random.randn(NH, NO) * std    # bobot hidden-output
    wb_weights = np.random.randn(NO) * std   # bobot bias-output

    return v_weights, vb_weights, w_wights, wb_weights

In [21]:
v, vb, w, wb = weightsd(NI, NH, NO)

print("Rentang: ", v.min(), v.max())
print("v =\n", v, "\n", np.shape(v))

print("\nRentang: ", vb.min(), vb.max())
print("vb =\n", vb, "\n", np.shape(vb))

print("\nRentang: ", w.min(), w.max())
print("w =\n", w, "\n", np.shape(w))

print("\nRentang: ", wb.min(), wb.max())
print("wb =\n", wb, "\n", np.shape(wb))

Rentang:  -0.1527948077200883 0.1816195042088606
v =
 [[ 0.02341533 -0.00651784  0.03053233]
 [ 0.07179632 -0.0110381  -0.01103732]
 [ 0.07444481  0.03617722 -0.02213123]
 ...
 [-0.01150606 -0.00185296 -0.00634024]
 [ 0.01573831  0.06747528  0.05099497]
 [-0.06185861  0.02932466  0.06264652]] 
 (900, 3)

Rentang:  0.018234328533338142 0.0948593385157525
vb =
 [0.01823433 0.05142931 0.09485934] 
 (3,)

Rentang:  -0.11924460562376078 0.09367095911431697
w =
 [[ 0.04825813  0.01175252  0.04926594  0.00683008  0.0011268  -0.01656583
   0.07370001 -0.0385811   0.07223651  0.02356382 -0.06596393  0.01733115
  -0.0989962   0.02948673  0.04173021]
 [-0.02792393  0.00582409  0.09211986 -0.02384113 -0.04991738  0.06984628
   0.09251725  0.00017422  0.04768082  0.06323778 -0.03500104 -0.02287754
   0.05802398  0.07943233  0.02653446]
 [-0.04147485  0.09367096 -0.02503021 -0.01579855  0.01613796  0.07326382
   0.04025681  0.01955695  0.02183963  0.0020513   0.02631095 -0.11924461
  -0.01398221  0.

**BPNN**

Feedforward

In [22]:
def feed_forward(x, v, vb, w, wb):
    # sinyal unit hidden
    z_in = np.dot(x, v) + vb   # sinyal input unit hidden
    z = relu(z_in)             # sinyal output unit hidden

    # sinyal unit output
    y_in = np.dot(z, w) + wb   # sinyal input unit output
    y = softmax(y_in)          # sinyal output unit output

    return z_in, z, y_in, y

Backpropagation Error

In [23]:
def backPropagation_error(x, t, z, z_in, y, y_in, v, vb, w, wb, NH, NO):
    # error unit output
    delta_y = (t - y) * d_softmax(y_in)

   # koreksi bobot unit output
    del_w = np.zeros((NH, NO))
    for j in range(NH):
        for k in range(NO):
            del_w[j][k] = lr * delta_y[k] * z[j]

    # koreksi bobot bias unit output
    del_wb = lr * delta_y

    # error unit hidden
    delta_zin = np.zeros(NH)
    for j in range(NH):
        for k in range(NO):
            delta_zin[j] += delta_y[k] * w[j][k]

    # aktivasi koreksi error unit hidden
    delta_z = delta_zin * d_relu(z_in)

    # koreksi bobot unit hidden
    del_v = np.zeros((NI, NH))
    for i in range(NI):
        for j in range(NH):
            del_v[i][j] = lr * delta_z[j] * x[i]

    # koreksi bobot bias unit hidden
    del_vb = lr * delta_z

    # update bobot dan bias unit output
    w += del_w
    wb += del_wb

    # update bobot dan bias unit hedden
    v += del_v
    vb += del_vb

    return v, vb, w, wb

**Implementasi training data**

In [24]:
for e in range(epoch):
    total_error_cross_entropy = 0

    for i in range(len(data)):
        x = data[i]
        t = labels[i]

        # Proses Feedforward
        z_in, z, y_in, y = feed_forward(x, v, vb, w, wb)

        # Proses Backpropagation
        v, vb, w, wb = backPropagation_error(x, t, z, z_in, y, y_in, v, vb, w, wb, NH, NO)

        # Error cross-entropy
        cross_entropy = -np.sum(t * np.log(y) + (1 - t) * np.log(1 - y))
        total_error_cross_entropy += cross_entropy

    avg_error = total_error_cross_entropy / len(data)
    print(f"Epoch {e+1}/{epoch}, Error: {avg_error}")

Epoch 1/100, Error: 11.364752227758887
Epoch 2/100, Error: 21.093672425469386
Epoch 3/100, Error: 37.58795962556874
Epoch 4/100, Error: 22.344885233104634
Epoch 5/100, Error: 5.106360740339447
Epoch 6/100, Error: 3.2733088048952768
Epoch 7/100, Error: 3.382222885559831
Epoch 8/100, Error: 3.3882167148522124
Epoch 9/100, Error: 3.387809550299889
Epoch 10/100, Error: 9.389879174727986
Epoch 11/100, Error: 4.361968255672068
Epoch 12/100, Error: 3.3848548526769884
Epoch 13/100, Error: 3.3866960002256934
Epoch 14/100, Error: 3.3876976566108326
Epoch 15/100, Error: 3.387732460938342
Epoch 16/100, Error: 3.387728368251696
Epoch 17/100, Error: 3.3877278992400957
Epoch 18/100, Error: 3.3877278899403627
Epoch 19/100, Error: 3.3877278919096523
Epoch 20/100, Error: 3.3877278921052905
Epoch 21/100, Error: 3.3877278921086718
Epoch 22/100, Error: 3.3877278921078284
Epoch 23/100, Error: 3.387727892107753
Epoch 24/100, Error: 3.3877278921077516
Epoch 25/100, Error: 3.387727892107753
Epoch 26/100, Error

##**Hasil**

#**CONCLUSION**
---
